In [ ]:
#===============================================================
# DEBUG TOGGLE SWITCH
def d(*args, **kwargs):
    if d.ON:
        print(*args, **kwargs)
d.ON = True
#===============================================================

#================================================
if __name__ == "__main__":
  try:
    import segmentation_models_pytorch as smp
  except ImportError:
    !pip install -q segmentation_models_pytorch timm
    import segmentation_models_pytorch as smp
#============================================================0

#===============================================================
# IMPORTS
import os
import numpy as np
import torch
import torch.nn.functional as F
from PIL import Image
import pandas as pd
from sklearn.metrics import precision_score, recall_score, f1_score, jaccard_score, roc_auc_score, roc_curve
from torchvision import transforms as T
from tqdm import tqdm
import random
#=================================================================================================================




#=====================================================================================================================
# PATHS SETUP
root_dir = "/content/drive/MyDrive/MAGISTRALE/ANNO 1/Computer Vision/Project/RoadObstacleDetection"
frames_dir = os.path.join(root_dir, "Datasets/LostAndFound/leftImg8bit/test")
gt_dir = os.path.join(root_dir, "Datasets/LostAndFound/gtCoarse/test")
split_file = os.path.join(root_dir, "ProjectWorkspace/splits/lostAndFound_splits_clean.txt")
ckpt_path = os.path.join(root_dir, "ProjectWorkspace/ckpts/updated_loss_model.pth")
lambda_path = os.path.join(root_dir, "ProjectWorkspace/src/eval/lambda_hat_3rdattempt.txt")
output_csv = os.path.join(root_dir, "ProjectWorkspace/eval/lostAndFound_metrics.csv")
#=====================================================================================================

#==================================================================================
# NAVIGATION TO GET TO THE FOLDER WITH STUFF INSIDE
import sys
sys.path.append("/content/drive/MyDrive/MAGISTRALE/ANNO 1/Computer Vision/Project/RoadObstacleDetection/ProjectWorkspace/src")

#==============================================================================
# MODEL LOADING
from network.deeplab_dualhead import get_model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = get_model().to(device)
model.load_state_dict(torch.load(ckpt_path, map_location = device))
model.eval()
#=================================================================================



#==============================================================
# CALIB LOADING
with open(lambda_path, "r") as f:
  lambda_hat = float(f.read().strip())
d(f"Loaded lambda_hat = {lambda_hat}")
#===============================================================

#===================================================================================
# PREPROCESSING
transform = T.Compose([
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225])
])
#==================================================================================0


#===================================================================================
# INFERENCE
threshold_objectness = 0.51
results = []
all_gt_pixels = []
all_scores = []

min_pixels = 200 #EDIT: since the dataset is for several purposes, they added some very far obstacles (can't even see them they're at the vanishing point)
skipped_small_gt = 0

with open(split_file, "r") as f:
  rel_paths = f.read().splitlines()

for rel_path in tqdm(rel_paths):
  rgb_path = os.path.join(frames_dir, rel_path + "_leftImg8bit.png")
  label_path = os.path.join(gt_dir, rel_path + "_gtCoarse_labelIds.png")

  rgb_img = Image.open(rgb_path).convert("RGB")
  input_tensor = transform(rgb_img).unsqueeze(0).to(device)

  with torch.no_grad():
    seg_logits, obj_logits = model(input_tensor)
    seg_logits = F.interpolate(seg_logits, size=rgb_img.size[::-1], mode="bilinear", align_corners=False)
    obj_logits = F.interpolate(obj_logits, size=rgb_img.size[::-1], mode="bilinear", align_corners=False)

    softmax = torch.softmax(seg_logits, dim=1)
    objectness = torch.sigmoid(obj_logits)

    conf_score, _ = torch.max(softmax, dim=1)
    nonconformity = 1.0 - conf_score
    unknown_mask = (nonconformity > lambda_hat).squeeze(0).cpu().numpy()
    object_mask = (objectness > threshold_objectness).squeeze().cpu().numpy()
    obstacle_mask = (unknown_mask & object_mask).astype(np.uint8)
    obstacle_score = (objectness.squeeze().cpu().numpy()) * (nonconformity.squeeze().cpu().numpy())

  hazard_ids = list(range(2, 44)) # from 2 to 43, doesn't take 44
  gt_mask = np.array(Image.open(label_path))
  gt_obstacle = np.isin(gt_mask, hazard_ids).astype(np.uint8)

  num_gt = np.sum(gt_obstacle)
  if num_gt < min_pixels:
    skipped_small_gt += 1
    continue

  pred_flat = obstacle_mask.flatten()
  gt_flat = gt_obstacle.flatten()
  score_flat = obstacle_score.flatten()

  precision = precision_score(gt_flat, pred_flat, zero_division=0)
  recall = recall_score(gt_flat, pred_flat, zero_division=0)
  f1 = f1_score(gt_flat, pred_flat, zero_division=0)
  iou = jaccard_score(gt_flat, pred_flat, zero_division=0)

  results.append({
      "image": rel_path,
      "precision": precision,
      "recall": recall,
      "f1": f1,
      "iou": iou
  })


  all_gt_pixels.append(gt_flat)
  all_scores.append(score_flat)

df = pd.DataFrame(results)
df.to_csv(output_csv, index=False)
d(f"Saved metrics to {output_csv}")
print(f"Total skipped images due to small GT obstacles or totally absent: {skipped_small_gt}")
#================================================================================0


#=================================================================================
# GLOBAL AGGREGATED METRICS
metrics = ["precision", "recall", "f1", "iou"]
global_stats = df[metrics].agg(["mean", "std", "min", "max", "median"])
print("\n== AGGREGATED METRICS ==")
print(global_stats)
#========================================================================================



#=================================================================================================
# GLOBAL SCORE-BASED METRICS (Filtered Random Subset to avoid OOM)
filtered_entries = [
    (gt, sc) for (gt, sc) in zip(all_gt_pixels, all_scores)
    if np.sum(gt) >= min_pixels
]

random.shuffle(filtered_entries)
subset_size = 200  # reduce memory load
filtered_entries = filtered_entries[:subset_size]

if len(filtered_entries) > 0:
    sampled_gt = np.concatenate([gt for gt, _ in filtered_entries])
    sampled_score = np.concatenate([sc for _, sc in filtered_entries])

    auroc = roc_auc_score(sampled_gt, sampled_score)
    fpr, tpr, _ = roc_curve(sampled_gt, sampled_score)
    fpr95 = fpr[np.argmax(tpr >= 0.95)]

    print("\n== GLOBAL SCORE-BASED METRICS (Filtered Subset) ==")
    print(f"AUROC = {auroc:.6f}")
    print(f"FPR@95TPR = {fpr95:.6f}")
else:
    print("\n[WARNING] No valid samples found for AUROC/FPR95 computation.")
#==================================================================================================


# FOR PRESENTATION
q_worst = df["iou"].quantile(0.30)
q_top = df["iou"].quantile(0.90)

worst_samples = set(df[df["iou"] <= q_worst]["image"])
top_samples = set(df[df["iou"] >= q_top]["image"])


In [ ]:
#===============================================================================================
# DISPLAY PREDICTIONS FOR PRESENTATION (same as RoadAnomaly, just gonna change the directory names)
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.colors import LinearSegmentedColormap

save_outputs = True # toggling until they're good to be saved

# Load CSV of per-image metrics
metrics_df = pd.read_csv(output_csv)

# Sort by IoU (assuming column name is 'iou' and image column is 'image')
sorted_df = metrics_df.sort_values(by='iou', ascending=False).reset_index(drop=True)

# Extract file paths
top_samples = set(sorted_df.iloc[:10]["image"])
worst_samples = set(sorted_df.iloc[-10:]["image"])
middle_pool = sorted_df.iloc[10:-10]["image"].tolist()

# Sample 20 randomly from the middle pool
random.seed(42)  # for reproducibility
middle_samples = set(random.sample(middle_pool, 20))

# Set of samples to actually save
selected_samples = top_samples.union(worst_samples).union(middle_samples)

save_dir_root = os.path.join(root_dir, "ProjectWorkspace/eval/LostAndFoundInferenceImages")
save_top_root = os.path.join(save_dir_root, "TopResults")
save_worst_root = os.path.join(save_dir_root, "WorstResults")
os.makedirs(save_dir_root, exist_ok=True)
os.makedirs(save_top_root, exist_ok=True)
os.makedirs(save_worst_root, exist_ok=True)


for sample in results:
    rel_path = sample["image"]
    rgb_path = os.path.join(frames_dir, rel_path + "_leftImg8bit.png")
    label_path = os.path.join(gt_dir, rel_path + "_gtCoarse_labelIds.png")
    name = os.path.basename(rel_path)

    is_top = rel_path in top_samples
    is_worst = rel_path in worst_samples
    should_save = save_outputs and (rel_path in selected_samples)

    # DEBUG: informazioni di base
    d("→ Processing:", rel_path)
    d("   is_top:", is_top, "is_worst:", is_worst)
    d("   selected_samples contains rel_path:", rel_path in selected_samples)
    d("   should_save:", should_save)
    d("   RGB exists:", os.path.exists(rgb_path))
    d("   LABEL exists:", os.path.exists(label_path))

    if not os.path.exists(rgb_path) or not os.path.exists(label_path):
        d("Missing files for", rel_path)
        continue

    rgb_img = Image.open(rgb_path).convert("RGB")

    hazard_ids = list(range(2, 44))  # Class IDs from 2 to 43 inclusive
    gt_mask = np.array(Image.open(label_path))
    gt_obstacle = np.isin(gt_mask, hazard_ids).astype(np.uint8)
    pixel_count = np.sum(gt_obstacle)
    d("   GT Obstacle pixel count:", pixel_count)

    if pixel_count < min_pixels:
        d("Skipping due to insufficient GT pixels")
        continue

    input_tensor = transform(rgb_img).unsqueeze(0).to(device)
    with torch.no_grad():
        seg_logits, obj_logits = model(input_tensor)
        seg_logits = F.interpolate(seg_logits, size=rgb_img.size[::-1], mode="bilinear", align_corners=False)
        obj_logits = F.interpolate(obj_logits, size=rgb_img.size[::-1], mode="bilinear", align_corners=False)

        softmax = torch.softmax(seg_logits, dim=1)
        objectness = torch.sigmoid(obj_logits)
        conf_score, _ = torch.max(softmax, dim=1)
        nonconformity = 1.0 - conf_score

        unknown_mask = (nonconformity > lambda_hat).squeeze(0).cpu().numpy()
        object_mask = (objectness > threshold_objectness).squeeze().cpu().numpy()
        obstacle_mask = (unknown_mask & object_mask).astype(np.uint8)

        conf_map = conf_score.squeeze().cpu().numpy()
        obstacle_score = objectness.squeeze().cpu().numpy() * nonconformity.squeeze().cpu().numpy()

    if should_save:
        if is_top:
            save_dir = os.path.join(save_top_root, name)
        elif is_worst:
            save_dir = os.path.join(save_worst_root, name)
        else:
            save_dir = os.path.join(save_dir_root, name)
        os.makedirs(save_dir, exist_ok=True)

        d("Saving in:", save_dir)

        rgb_img.save(os.path.join(save_dir, "rgb.png"))

        fig, ax = plt.subplots(figsize=(6, 5))
        im = ax.imshow(conf_map, cmap="viridis", vmin=0.0, vmax=1.0)
        ax.axis("off")
        cbar = plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
        cbar.set_label("Confidence scale")
        plt.savefig(os.path.join(save_dir, "confidence_map.png"), dpi=200)
        plt.close()

        fig, ax = plt.subplots(figsize=(6, 5))
        im = ax.imshow(obstacle_score, cmap="viridis_r", vmin=0.0, vmax=1.0)
        ax.axis("off")
        cbar = plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
        cbar.set_label("Obstacle Score")
        plt.savefig(os.path.join(save_dir, "obstacle_score.png"), dpi=200)
        plt.close()

        if is_top or (not is_worst):
            Image.fromarray((gt_obstacle * 255).astype(np.uint8)).save(os.path.join(save_dir, "gt_obstacle.png"))
            Image.fromarray((unknown_mask * 255).astype(np.uint8)).save(os.path.join(save_dir, "unknown_mask.png"))
            Image.fromarray((object_mask * 255).astype(np.uint8)).save(os.path.join(save_dir, "object_mask.png"))
            Image.fromarray((obstacle_mask * 255).astype(np.uint8)).save(os.path.join(save_dir, "obstacle_mask.png"))


In [ ]:
""" #kept here for backup
  fig, axs = plt.subplots(1, 7, figsize=(42, 6))
  axs[0].imshow(rgb_img)
  axs[0].set_title("RGB Image")
  axs[1].imshow(gt_obstacle, cmap="gray")
  axs[1].set_title("GT Obstacle")
  axs[2].imshow(unknown_mask, cmap="gray")
  axs[2].set_title("Unknown Mask")
  axs[3].imshow(object_mask, cmap="gray")
  axs[3].set_title("Objectness Mask")
  axs[4].imshow(obstacle_mask, cmap="gray")
  axs[4].set_title("Obstacle = Unknown AND Object")
  # CONFIDENCE MAP USING ONLY SOFTMAX
  im1 = axs[5].imshow(conf_map, cmap="viridis", vmin=0.0, vmax=1.0)
  axs[5].set_title("Confidence Map (Max Softmax)")
  cbar1 = fig.colorbar(im1, ax=axs[5], fraction=0.046, pad=0.04)
  cbar1.set_label("Confidence scale")
  # CONFIDENCE MAP USING SOFTMAX AND SIGMOID HEAD COMBINED
  im2 = axs[6].imshow(obstacle_score, cmap="viridis_r", vmin=0.0, vmax=1.0) #doing viridis"_r" in order to revert the behaviour of the viridis heatmap and return an image prone to be compared to the other confidence map
  axs[6].set_title("Obstacle Score = Objectness combined with Nonconformity")
  cbar2 = fig.colorbar(im2, ax=axs[6], fraction=0.046, pad=0.04)
  cbar2.set_label("Obstacle Score")

  for ax in axs:
      ax.axis("off")
  plt.suptitle(name, fontsize=18)
  plt.tight_layout()
  plt.show()
"""